# * Import needed modules, for local and for Colab

In [1]:
# Import system libs
import os
import time
import shutil
import pathlib
import itertools

# Import data handling libs
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Import machine learning libs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Load dataset as generators

In [2]:
TRAIN_DIR = f'./DATASET/train'
VALIDATION_DIR = f'./DATASET/validation'
TEST_DIR = f'./DATASET/test'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

"""
On this project, generator were used to handle RAM usage crash.
It basically load one batch at a time from the dataframe instead of loading all at once
to speedup training.
"""

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
LABEL_MODE = 'categorical'
COLOR_MODE = 'rgb'  # accepts 'grayscale', 'rgb', and 'rgba' only
SEED = 42

def preprocess(image):
    """
    Preprocessing function: Resize and normalize the images
    PREPROCESS_INPUT_MODE_DOC =
    mode: One of "caffe", "tf" or "torch".
      - caffe: will convert the images from RGB to BGR,
          then will zero-center each color channel with
          respect to the ImageNet dataset,
          without scaling.
      - tf: will scale pixels between -1 and 1,
          sample-wise.
      - torch: will scale pixels between 0 and 1 and then
          will normalize each channel with respect to the
          ImageNet dataset.
      Defaults to `"caffe"`.

      Note: each Keras Application expects a specific kind of input preprocessing.
      For MobileNetV3, by default input preprocessing is included as a part of the
      model (as a `Rescaling` layer), and thus
      `keras.applications.mobilenet_v3.preprocess_input` is actually a
      pass-through function. In this use case, MobileNetV3 models expect their
      inputs to be float tensors of pixels with values in the `[0-255]` range.
      At the same time, preprocessing as a part of the model (i.e. `Rescaling`
      layer) can be disabled by setting `include_preprocessing` argument to `False`.
      With preprocessing disabled MobileNetV3 models expect their inputs to be float
      tensors of pixels with values in the `[-1, 1]` range.
    """
    # # image = tf.image.resize(image, (224, 224), preserve_aspect_ratio=True)  # Resize to 224x224
    # image = tf.cast(image, tf.float32)  # Convert to float32
    # image = (image / 127.5) - 1  # Normalize to [-1, 1] range
    return image

def create_gens():
    """ Function to load from pre-split dataset directories using ImageDataGenerator with preprocessing """
    # Test Batch Size
    

    # Initialize the ImageDataGenerator without validation split but with preprocessing function
    tr_gen = ImageDataGenerator(
        preprocessing_function=preprocess,
        horizontal_flip=True
    )
    ts_gen = ImageDataGenerator(
        preprocessing_function=preprocess
    )

    # Train Generator (from the pre-split training directory)
    train_generator = tr_gen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=LABEL_MODE,
        color_mode=COLOR_MODE,
        seed=SEED,
        shuffle=True
    )

    # Validation Generator (from the pre-split validation directory)
    validation_generator = ts_gen.flow_from_directory(
        VALIDATION_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=LABEL_MODE,
        color_mode=COLOR_MODE,
        seed=SEED,
        shuffle=True
    )

    # Test Generator (from the pre-split test directory)
    test_generator = ts_gen.flow_from_directory(
        TEST_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=LABEL_MODE,
        color_mode=COLOR_MODE,
        seed=SEED,
        shuffle=False
    )

    return train_generator, validation_generator, test_generator

In [4]:
""" Create datasets """
train_gen, val_gen, test_gen = create_gens()

Found 14814 images belonging to 4 classes.
Found 4252 images belonging to 4 classes.
Found 2099 images belonging to 4 classes.


In [5]:
print(train_gen.class_indices)

{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral Pneumonia': 3}


# Evaluate Model

In [6]:
MODEL_PATH = f'MODELS\mobilenetv3small\mobilenetv3small-COVID-19-94.90.keras'
model = tf.keras.models.load_model(MODEL_PATH)

In [7]:
ts_length = test_gen.samples
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

# train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
# valid_score = model.evaluate(val_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

# print("Train Loss: ", train_score[0])
# print("Train Accuracy: ", train_score[1])
# print('-' * 20)
# print("Validation Loss: ", valid_score[0])
# print("Validation Accuracy: ", valid_score[1])
# print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

2099/2099 ━━━━━━━━━━━━━━━━━━━━ 55s 24ms/step - accuracy: 0.9489 - loss: 0.2835   
Test Loss:  0.28275734186172485
Test Accuracy:  0.9490233659744263
